In [1]:
import xml.etree.ElementTree as ET
from pathlib import Path
import datetime as dt
import xmltodict
import pandas as pd
import os

In [2]:
def read_eml(path: Path):

    """Convert EML file to dictionary"""

    try:
        return xmltodict.parse(path.read_text(encoding="utf-8"))

    except UnicodeDecodeError as e:
        print(path.name)
        print(e)
        return None

In [5]:
import os

# Get the user's home directory
home_directory = os.path.expanduser("~")

# Construct the full path to your file
file_path = os.path.join(home_directory, "aec-mediafeed-Detailed-Verbose-29581-20230920135814/xml/aec-mediafeed-results-detailed-verbose-29581.xml")

f = open(file_path, encoding='utf-8')
text = f.read()
temp = xmltodict.parse(text)

In [29]:

PollingDistrict =temp['MediaFeed']['Results']['Election']['Referendum']['Contests']['Contest']['PollingDistricts']['PollingDistrict'][0]
PollingDistrict

{'PollingDistrictIdentifier': {'@Id': '318',
  '@ShortCode': 'BEAN',
  'Name': 'Bean',
  'StateIdentifier': {'@Id': 'ACT'}},
 'Enrolment': {'@CloseOfRolls': '109557',
  '@Historic': '109432',
  '#text': '109557'},
 'ProposalResults': {'@PollingPlacesReturned': '0',
  '@PollingPlacesExpected': '49',
  'Option': [{'eml:ReferendumOptionIdentifier': {'@Id': 'Y', '#text': 'Yes'},
    'Votes': {'@Percentage': '0', '#text': '0'},
    'VotesByType': {'Votes': [{'@Type': 'Ordinary',
       '@Percentage': '0',
       '#text': '0'},
      {'@Type': 'Absent', '@Percentage': '0', '#text': '0'},
      {'@Type': 'Provisional', '@Percentage': '0', '#text': '0'},
      {'@Type': 'PrePoll', '@Percentage': '0', '#text': '0'},
      {'@Type': 'Postal', '@Percentage': '0', '#text': '0'}]}},
   {'eml:ReferendumOptionIdentifier': {'@Id': 'N', '#text': 'No'},
    'Votes': {'@Percentage': '0', '#text': '0'},
    'VotesByType': {'Votes': [{'@Type': 'Ordinary',
       '@Percentage': '0',
       '#text': '0'},
  

In [33]:
df = pd.DataFrame()

for PollingDistrict in temp['MediaFeed']['Results']['Election']['Referendum']['Contests']['Contest']['PollingDistricts']['PollingDistrict']:
    temp_dict = {
        'Seat': PollingDistrict['PollingDistrictIdentifier']['Name'],
        'SeatID': PollingDistrict['PollingDistrictIdentifier']['@Id'],
        'State': PollingDistrict['PollingDistrictIdentifier']['StateIdentifier']['@Id'],
        'Enrolment': float(PollingDistrict['Enrolment']['#text']),
        'Yes': float(PollingDistrict['ProposalResults']['Option'][0]['Votes']['#text']),
        'No': float(PollingDistrict['ProposalResults']['Option'][1]['Votes']['#text']),
        'Informal': float(PollingDistrict['ProposalResults']['Informal']['Votes']['#text']),
        'Complete': PollingDistrict['ProposalResults']['@PollingPlacesExpected'] == PollingDistrict['ProposalResults']['@PollingPlacesReturned'],
        'Votes_returned': float(PollingDistrict['ProposalResults']['Total']['Votes']['@Percentage'])
    }
    temp_dict['Informal_pc'] = temp_dict['Informal'] / temp_dict['Enrolment']
    temp_dict['Yes_pc'] = temp_dict['Yes'] / temp_dict['Enrolment']
    temp_dict['No_pc'] = temp_dict['No'] / temp_dict['Enrolment']
    df = pd.concat(
        [df, pd.DataFrame(temp_dict, index=[0])], ignore_index=True
    )